In [14]:
# Импортируем библиотеку для выполнения HTTP-запросов в интернет
import requests
from math import log

# Читаем текстовый файл по url-ссылке
data = requests.get("https://raw.githubusercontent.com/SkillfactoryDS/Datasets/master/war_peace_processed.txt").text

# Предобрабатываем текстовый файл
data = data.split('\n')
data.remove('')
data = data + ['[new chapter]']

# Выводим первые 100 слов из книги
print(data[:100])

['1', 'в', 'два', 'раза', 'короче', 'и', 'в', 'пять', 'раз', 'интереснее', '2', 'почти', 'нет', 'философических', 'отступлений', '3', 'в', 'сто', 'раз', 'легче', 'читать', 'весь', 'французский', 'текст', 'заменен', 'русским', 'в', 'переводе', 'самого', 'толстого', '4', 'гораздо', 'больше', 'мира', 'и', 'меньше', 'войны', '5', 'хеппи-энд', 'эти', 'слова', 'я', 'поместил', 'семь', 'лет', 'назад', 'на', 'обложку', 'предыдущего', 'издания', 'указав', 'в', 'аннотации', 'первая', 'полная', 'редакция', 'великого', 'романа', 'созданная', 'к', 'концу', '1866', 'года', 'до', 'того', 'как', 'толстой', 'переделал', 'его', 'в', '1867--1869', 'годах', '--', 'и', 'что', 'я', 'использовал', 'такие-то', 'публикации', 'думая', 'что', 'все', 'всё', 'знают', 'я', 'не', 'объяснил', 'откуда', 'взялась', 'эта', 'первая', 'редакция', 'я', 'оказался', 'неправ', 'и', 'в', 'результате', 'оголтелые', 'и']


In [15]:
# Разделяем текст на главы
chapter_data = []
chapter_words = []

for word in data:
    if word == '[new chapter]':
        chapter_data.append(chapter_words)
        chapter_words = []
    else:
        chapter_words.append(word)

In [16]:
# Инициализируем список для хранения количества слов в каждой главе
chapter_words_count = []

# Считаем количество слов в каждой главе
for chapter_words in chapter_data:
    temp = {}
    for word in chapter_words:
        if word not in temp:
            temp[word] = 1
        else:
            temp[word] += 1
    chapter_words_count.append(temp)

# Проверяем, что список chapter_words_count определен
print("Список chapter_words_count определен и содержит", len(chapter_words_count), "элементов.")

# Считаем общее количество слов в каждой главе
chapter_total_words = []
for chapter in chapter_data:
    chapter_total_words.append(len(chapter))

Список chapter_words_count определен и содержит 171 элементов.


In [17]:
target_word = 'гостья'
target_chapter = 15

# Ваш код здесь

# Считаем частоту встречаемости слов (TF) для каждой главы
tf_per_chapter = []
for chapter_num in range(len(chapter_words_count)):
    chapter_dict = chapter_words_count[chapter_num]
    chapter_total = chapter_total_words[chapter_num]
    tf_chapter_dict = {}
    for word in chapter_dict:
        tf_chapter_dict[word] = chapter_dict[word] / chapter_total
    tf_per_chapter.append(tf_chapter_dict)

print(tf_per_chapter[target_chapter][target_word])

0.007358351729212656


In [18]:
target_word = 'человек'
# Считаем частоту документов (DF) для каждого слова
word_set = set()
for chapter in chapter_data:
    for word in chapter:
        word_set.add(word)

df_dict = {}
total_chapters = len(chapter_words_count)
for word in word_set:
    contain_word_chapters = 0
    for chapter_dict in chapter_words_count:
        if word in chapter_dict:
            contain_word_chapters += 1
    df_dict[word] = contain_word_chapters / total_chapters

print(df_dict[target_word])
# Ваш код здесь

0.672514619883041


In [42]:
target_word = 'анна'
target_chapter = 4
# Считаем нормализованное TF-IDF для каждого слова в каждой главе
tf_idf_per_chapter = []
for chapter_tf_dict in tf_per_chapter:
    tf_idf_chapter_dict = {}
    for word in chapter_tf_dict:
        if len(word) > 3 and df_dict[word] > 0:
            tf_idf_chapter_dict[word] = chapter_tf_dict[word] * log(1 / df_dict[word])
    tf_idf_per_chapter.append(tf_idf_chapter_dict)
print(tf_idf_per_chapter[target_chapter][target_word])
# Ваш код здесь

0.011067446769736353


In [ ]:
# Получаем топ-3 слова с наивысшим TF-IDF в заданной главе
target_chapter = 170  # Измените на нужный номер главы
if target_chapter <= len(tf_idf_per_chapter):
    chapter_tf_idf_dict = tf_idf_per_chapter[target_chapter]

    # Сортируем слова по значению TF-IDF в порядке убывания
    sorted_tf_idf = sorted(chapter_tf_idf_dict.items(), key=lambda item: item[1], reverse=True)

    # Печатаем топ-3 слова
    print("Топ-3 слова с наивысшим TF-IDF в главе", target_chapter)
    for word, tf_idf in sorted_tf_idf[:3]:
        print(word, tf_idf)
else:
    print("Ошибка: Номер главы превышает количество глав в тексте.")

